<h1 style="text-align: center;">Data Mining</h1>
<h2 style="text-align: center;">Exercise Sheet 3: Causality</h2>
<h3 style="text-align: center;">Iftekher Aziz - 12338137</h3>

-------

# Exercise 3-1 Granger causal test:

## (a): Auto-regression for X and Y

In [5]:
import numpy as np

# Data: Chickens (Y) and Eggs (X)
Y = np.array([468491, 449743, 436815, 444523, 433937, 389958])
X = np.array([3581, 3532, 3327, 3255, 3156, 3081])

# Prepare the design matrix and target vector
lagged_Y = Y[:-1]  # Lagged chicken population
lagged_X = X[:-1]  # Lagged egg production
target_Y = Y[1:]   # Chicken population (next time step)

# Build the design matrix
design_matrix = np.column_stack((np.ones(len(lagged_Y)), lagged_Y, lagged_X))

# Compute coefficients using the formula beta = (X^T X)^-1 X^T Y
XtX = np.dot(design_matrix.T, design_matrix)  # X^T * X
XtY = np.dot(design_matrix.T, target_Y)       # X^T * Y
beta = np.linalg.solve(XtX, XtY)             # Solve for beta

# Compute residuals and RSS1
predicted_Y = np.dot(design_matrix, beta)    # Predicted Y values
residuals = target_Y - predicted_Y           # Residuals
RSS1 = np.sum(residuals ** 2)                # Residual Sum of Squares

# Output results
print("Computed Coefficients:")
print(f"β0 (Intercept): {beta[0]:.2f}")
print(f"β1 (Lagged Y effect): {beta[1]:.2e}")
print(f"β2 (Lagged X effect): {beta[2]:.2f}")

print("\nModel Performance:")
print(f"Residual Sum of Squares (RSS1): {RSS1:.2f}")

Computed Coefficients:
β0 (Intercept): 120111.88
β1 (Lagged Y effect): -6.90e-02
β2 (Lagged X effect): 101.40

Model Performance:
Residual Sum of Squares (RSS1): 1023104023.61


----

#### **Computed Coefficients:**
- **β0 (Intercept): 120111.88**  
  This is the intercept, representing the baseline chicken population (\(Y\)) when the effects of lagged \(Y\) and \(X\) are zero. 

- **β1 Lagged Y effect): -6.90e-02**  
  This coefficient represents the influence of the lagged chicken population (\(Y\)) on the current chicken population. The negative value indicates a slight decrease in the current chicken population as the lagged chicken population increases.

- **β2 (Lagged X effect): 101.40**  
  This coefficient reflects the effect of the lagged egg production (\(X\)) on the current chicken population. A positive value suggests that higher egg production in the previous period contributes to an increase in the chicken population.

#### **Model Performance:**
- **Residual Sum of Squares (RSS1) = 1023104023.61**  
  This value quantifies the discrepancy between the observed and predicted chicken populations. A smaller RSS value implies a better model fit. In this case, the relatively large RSS suggests significant variability in the data remains unexplained by the model.


----

## (b): Auto-regression without Y

In [10]:
# Design matrix without lagged Y: [Intercept, lagged_X]
design_matrix_reduced = np.column_stack((np.ones(len(lagged_X)), lagged_X))

# Compute beta using the formula: beta = (X^T X)^-1 X^T Y
XtX_reduced = np.dot(design_matrix_reduced.T, design_matrix_reduced)
XtY_reduced = np.dot(design_matrix_reduced.T, target_Y)
beta_reduced = np.linalg.solve(XtX_reduced, XtY_reduced)  # Solve for beta

# Compute residuals and RSS
residuals_reduced = target_Y - np.dot(design_matrix_reduced, beta_reduced)
RSS2 = np.sum(residuals_reduced ** 2)


# Output results
print("Computed Coefficients without Lagged Y:")
print(f"β0 (Intercept): {beta_reduced[0]:.2f}")
print(f"β2 (Lagged X effect): {beta_reduced[1]:.2f}")

print("\nModel Performance:")
print(f"Residual Sum of Squares (RSS2): {RSS2:.2f}")

Computed Coefficients without Lagged Y:
β0 (Intercept): 104291.23
β2 (Lagged X effect): 96.94

Model Performance:
Residual Sum of Squares (RSS2): 1024061715.71


-----

#### **Computed Coefficients:**
- **β0 (Intercept): 104291.23**  
  This is the intercept, representing the base value of the chicken population (\(Y\)) when the effect of lagged egg production (\(X\)) is zero.

- **β2 (Lagged X effect): 96.94**  
  This coefficient represents the effect of lagged egg production (\(X\)) on the current chicken population (\(Y\)). The positive value indicates that an increase in lagged egg production contributes to an increase in the chicken population.

#### **Model Performance:**
- **Residual Sum of Squares (RSS2): 1024061715.71**  
  This value quantifies the discrepancy between the observed and predicted chicken populations when only lagged egg production (X) is considered. RSS2 is slightly higher than RSS1 (from part (a), indicating that excluding lagged chicken population (Y) reduces the model's ability to explain the variation in (Y).

-----

## (c): Apply statistical test

In [15]:
from scipy.stats import f

# Given values
n = len(target_Y)  # Number of observations in the target vector
k = design_matrix.shape[1]  # Number of parameters in the full model (intercept, lagged_Y, lagged_X)
p = 1  # Number of parameters removed (lagged_Y)

# F-statistic formula
F = ((RSS2 - RSS1) / p) / (RSS1 / (n - k))

# Critical value for F-distribution at alpha = 0.05
alpha = 0.05
F_critical = f.ppf(1 - alpha, p, n - k)

# Decision
if F > F_critical:
    print("Reject the null hypothesis: Lagged X (eggs) Granger-causes Y (chickens).")
else:
    print("Fail to reject the null hypothesis: No evidence that lagged X (eggs) Granger-causes Y (chickens).")

# Output results
print(f"\nF-statistic: {F:.4f} (Indicates ratio of explained variance)")
print(f"Critical value (F_critical): {F_critical:.4f} (Threshold for significance at alpha = 0.05)")

Fail to reject the null hypothesis: No evidence that lagged X (eggs) Granger-causes Y (chickens).

F-statistic: 0.0019 (Indicates ratio of explained variance)
Critical value (F_critical): 18.5128 (Threshold for significance at alpha = 0.05)


-----

**Since the F-statistic (0.0019) is much smaller than the critical value (18.5128), we fail to reject the null hypothesis. This means there is no significant evidence that lagged egg production (X) Granger-causes the chicken population (Y).**

-----

## (d): Causal test for direction from chicken to eggs

#### Regression with Lagged X and 𝑌

In [21]:
# Design matrix: [Intercept, lagged_X, lagged_Y]
design_matrix_X_full = np.column_stack((np.ones(len(lagged_X)), lagged_X, lagged_Y))

# Compute beta
XtX_full = np.dot(design_matrix_X_full.T, design_matrix_X_full)
XtY_full = np.dot(design_matrix_X_full.T, X[1:])  # Target is eggs (X) starting from the second row
beta_full = np.linalg.solve(XtX_full, XtY_full)

# Compute residuals and RSS1
residuals_full = X[1:] - np.dot(design_matrix_X_full, beta_full)
RSS1_X = np.sum(residuals_full ** 2)

#### Regression Without Lagged Y

In [23]:
# Design matrix without lagged_Y: [Intercept, lagged_X]
design_matrix_X_reduced = np.column_stack((np.ones(len(lagged_X)), lagged_X))

# Compute beta
XtX_reduced_X = np.dot(design_matrix_X_reduced.T, design_matrix_X_reduced)
XtY_reduced_X = np.dot(design_matrix_X_reduced.T, X[1:])
beta_reduced_X = np.linalg.solve(XtX_reduced_X, XtY_reduced_X)

# Compute residuals and RSS2
residuals_reduced_X = X[1:] - np.dot(design_matrix_X_reduced, beta_reduced_X)
RSS2_X = np.sum(residuals_reduced_X ** 2)

#### Granger-Sargent Test

In [25]:
# Compute F-statistic for direction chickens -> eggs
F_XY = ((RSS2_X - RSS1_X) / p) / (RSS1_X / (n - k))

# Critical value
F_critical_XY = f.ppf(1 - alpha, p, n - k)

# Decision
if F_XY > F_critical_XY:
    print("Reject the null hypothesis: Lagged Y (chickens) Granger-causes X (eggs).")
else:
    print("Fail to reject the null hypothesis: No evidence that lagged Y (chickens) Granger-causes X (eggs).")

# Output results
print(f"\nF-statistic (chickens -> eggs): {F_XY:.4f} (Indicates ratio of explained variance)")
print(f"Critical value (F_critical): {F_critical_XY:.4f} (Threshold for significance at alpha = 0.05)")

Fail to reject the null hypothesis: No evidence that lagged Y (chickens) Granger-causes X (eggs).

F-statistic (chickens -> eggs): 1.2354 (Indicates ratio of explained variance)
Critical value (F_critical): 18.5128 (Threshold for significance at alpha = 0.05)


----

**The Granger causality test for the direction **chickens → eggs** resulted in a failure to reject the null hypothesis, with an \( F \)-statistic of 1.235 and a critical value of 18.513. This indicates **no evidence** that lagged chicken populations Granger-cause egg production. If the test for the opposite direction (**eggs → chickens**) showed significance, it would suggest that **eggs came first**. Otherwise, the data does not provide a conclusive answer to the question.**

----

# Exercise 3-2 Multivariate Granger model:

## (a): Consistency of Adaptive Lasso for Multivariate Granger Model

In [31]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# Generate synthetic multivariate Granger causality data
np.random.seed(42)
n = 100  # Number of observations
lags = 2  # Number of lags

# Generate predictors (X1 and X2) and response (Y)
X1 = np.random.normal(0, 1, n)
X2 = np.random.normal(0, 1, n)
Y = 0.8 * X1[:-1] + 0.5 * X2[:-1] + np.random.normal(0, 0.1, n-1)

# Create lagged predictors
lagged_X1 = X1[:-1]  # Lag 1 for X1
lagged_X2 = X2[:-1]  # Lag 1 for X2

# Design matrix for regression
X = np.column_stack((lagged_X1, lagged_X2))
Y = Y  # Target response (Y)

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform adaptive lasso regression using LassoCV
lasso_model = LassoCV(cv=5, alphas=np.logspace(-3, 2, 50))
lasso_model.fit(X_scaled, Y)

# Extract coefficients
lasso_coefficients = lasso_model.coef_
selected_features = np.where(lasso_coefficients != 0)[0]

# Output results
print(f"Selected Features (Lagged Predictors): {selected_features + 1} (Indices of relevant lagged predictors)")
print(f"Lasso Coefficients: {lasso_coefficients} (Weights for the selected features)")
print(f"Optimal Regularization Parameter (alpha): {lasso_model.alpha_:.3f} (Chosen dynamically via cross-validation)")


# Test Consistency
# Check if non-zero coefficients correspond to true model structure
true_coefficients = [0.8, 0.5]  # True coefficients for lagged_X1 and lagged_X2
consistency = all(
    (abs(true_coefficients[i] - lasso_coefficients[i]) < 0.1)
    if i in selected_features
    else lasso_coefficients[i] == 0
    for i in range(len(true_coefficients))
)

print("\nModel is consistent:", consistency)



Selected Features (Lagged Predictors): [1 2] (Indices of relevant lagged predictors)
Lasso Coefficients: [0.74601654 0.47162009] (Weights for the selected features)
Optimal Regularization Parameter (alpha): 0.001 (Chosen dynamically via cross-validation)

Model is consistent: True


----

### Is the Problem Solved by This Model Consistent?

The **graphical Granger model** with **adaptive lasso** is consistent in this case because:

#### Correct Feature Selection:
- The model correctly identified \( X1 \) and \( X2 \) as the relevant lagged predictors for \( Y \).
- Irrelevant predictors were excluded, confirming no false positives.

#### Error Bounds:
- The difference between the true coefficients ( 0.8, 0.5 ) and the estimated coefficients is within the tolerance (less than 0.1).


#### Regularization Parameter:
- λ was chosen dynamically by `LassoCV`, ensuring a good balance between sparsity and model fit.
- While this does not explicitly enforce ( λ_n = n^3/2 ) the method generalizes well for smaller datasets.


----

## (b): What is the algorithm HMMLGA for and what are its hyperparameters?

#### **What is HMMLGA?**
HMMLGA (High-order Mixed Model Lasso Granger Algorithm) is an algorithm used for **high-dimensional Granger causality analysis**. It identifies causal relationships among multiple time series by leveraging **lasso regression** to enforce sparsity and select the most relevant lagged variables. It is particularly effective in high-dimensional settings, where the number of variables exceeds the number of observations.

#### **Hyperparameters of HMMLGA:**
1. **Regularization Parameter (λ))**:
   - Controls sparsity by penalizing small coefficients.
   - Larger (λ): More coefficients shrink to zero.

2. **Lag Order (d)**:
   - Determines how many previous time points (lags) to include as predictors.

3. **Stopping Criteria**:
   - Convergence threshold or maximum iterations to stop optimization.

4. **Penalty Weights**:
   - Adaptive lasso may use weights to adjust the penalty for different predictors.

****This algorithm helps uncover directional influences between variables in time series data while handling high-dimensional challenges effectively.****

----

# Exercise 3-3 Bivariate causal models on non-temporal data:

## (a) Why is the causal relationship between X and Y by bivariate additive noise models for the linearGaussian case non-identifiable? Explain

#### **Bivariate Additive Noise Model**:
   - The model assumes a causal relationship between two variables \( X \) and \( Y \):
     - \( Y = f(X) + Ny \), where \( Ny \) is noise independent of \( X \).
     - Or, \( X = g(Y) + Nx \), where \( Nx \) is noise independent of \( Y \).
   - f(⋅) and g(⋅) are deterministic functions.

#### **Linear-Gaussian Case**:
   - When f(⋅) and g(⋅) are linear functions:
     - Y= βX+Ny
     - X=γY+Nx,
   - Nx and Ny are Gaussian random variables.

#### **Why Non-identifiable?**

In the **linear-Gaussian case**, the joint distribution of \( X \) and \( Y \) is symmetric, where ρ is the correlation coefficient.

Because this joint distribution is symmetric,both causal directions X→Y and Y→X produce the same joint distribution, making them indistinguishable from one another.


#### **Intuition Behind Non-identifiability**
 
- **Symmetric Joint Distribution**: The correlation coefficient (\(\rho\)) does not change regardless of the causal direction.
- **Gaussian Noise**: Symmetry in Gaussian noise prevents distinguishing cause from effect.
- **No Asymmetry**: Identifiability relies on asymmetry, which is absent in linear-Gaussian cases.

---

## Solution to Exercise 3-3 (b)

### 1. Causal Relation Between X and Y:
Age (X) causes blood pressure (Y) because aging stiffens the cardiovascular system, leading to changes in blood pressure. Thus, X→Y is the causal direction.

### 2. Changing P(effect∣cause) Without Affecting P(cause):
In X→Y, P(Y∣X) can be modified by interventions like antihypertensive drugs, which lower blood pressure without altering age (P(X)).

### 3. Changing P(cause) Without Affecting P(effect∣cause):
Alter P(X) (age distribution) by selective sampling of different age groups. This leaves the physiological relationship P(Y∣X) (blood pressure given age) intact.

### 4. Anti-Causal Direction:
Assuming Y→X (blood pressure causes age) is biologically implausible:
Age cannot be changed by altering blood pressure (Y).
The intrinsic relationship between age and blood pressure makes it unrealistic to adjust P(X∣Y) independently of P(Y).

### Summary of Findings:
**Causal Relation:** X→Y (age causes blood pressure).

**Causal Direction:**
 - Change P(Y∣X): Use drugs to reduce blood pressure.
 - Change P(X): Hypothetically sample different age groups.

**Anti-Causal Direction:** Unrealistic due to intrinsic biological dependence between age and blood pressure.